In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

# === Оси времени ===
years = [2023, 2024]
months = ["Январь","Февраль","Март","Апрель","Май","Июнь",
          "Июль","Август","Сентябрь","Октябрь","Ноябрь","Декабрь"]

# === Сезонные сдвиги климата ===
month_temp_shift = {"Январь":-10,"Февраль":-8,"Март":-4,"Апрель":2,"Май":8,
                    "Июнь":12,"Июль":14,"Август":12,"Сентябрь":8,
                    "Октябрь":2,"Ноябрь":-4,"Декабрь":-8}
month_rain_shift = {"Январь":30,"Февраль":35,"Март":60,"Апрель":80,"Май":70,
                    "Июнь":40,"Июль":30,"Август":25,"Сентябрь":40,
                    "Октябрь":50,"Ноябрь":45,"Декабрь":35}

# === Регионы ===
# Изменил базовые температурные диапазоны по замечанию: ГБАО значительно холоднее,
# Хатлон — самая тёплая, РРП и Согдийская — промежуточные (настроено так, чтобы
# средние температуры стали ближе к желаемым 25/21/17/<=10 и т.д.).
regions = {
    "Хатлонская": {"zone":"Равнинная","temp":(18,28)},   # средняя ~23
    "Согдийская": {"zone":"Полугорная","temp":(14,24)},  # средняя ~19
    "ГБАО":        {"zone":"Горная","temp":(0,9)},       # средняя <10
    "РРП":         {"zone":"Разнообразная","temp":(16,26)} # средняя ~21
}

# === Доли по продуктам ===
shares = {
    "Пшеница":   {"Хатлонская":0.40,"Согдийская":0.35,"РРП":0.20,"ГБАО":0.05},
    "Овощи":     {"Хатлонская":0.45,"Согдийская":0.30,"РРП":0.20,"ГБАО":0.05},
    "Картофель": {"Хатлонская":0.25,"Согдийская":0.40,"РРП":0.25,"ГБАО":0.10},
    "Мясо":      {"Хатлонская":0.38,"Согдийская":0.32,"РРП":0.20,"ГБАО":0.10},
    "Молоко":    {"Хатлонская":0.40,"Согдийская":0.30,"РРП":0.20,"ГБАО":0.10},
}

# === Страновые ориентиры ===
country_targets = {
    "Пшеница":   {"area": (220_000, 260_000), "yield": (2.8, 3.5)},  # без изменений
    "Картофель": {"area": (50_000, 55_000),   "yield": (20, 22)},    # ↑ увеличил площадь
    "Овощи":     {"area": (80_000, 90_000),   "yield": (14, 18)},    # ↑ чуть выше урожайность
    "Мясо":      {"volume": (350_000, 400_000)},                     # ↑ до 380k среднее
    "Молоко":    {"volume": (1_100_000, 1_250_000)},                 # ↑ до 1.18M среднее
}

# === Урожайные месяцы ===
harvest_months = {
    "Пшеница": ["Июнь","Июль"],
    "Картофель": ["Май","Июнь","Сентябрь"],
    "Овощи": ["Июнь","Июль","Август","Сентябрь"],
    "Мясо": months,
    "Молоко": months
}

# === Базовые цены (сом/т) ===
base_prices = {
    "Пшеница": (5000, 6000),
    "Картофель": (6000, 8000),
    "Овощи": (4000, 6000),
    "Мясо": (60000, 80000),
    "Молоко": (8000, 10000),
}

# === Чувствительность цены к дефициту ===
price_alpha = {
    "Пшеница": 0.25,
    "Картофель":0.20,
    "Овощи":   0.20,
    "Мясо":    0.15,
    "Молоко":  0.15,
}

# === Урожайность: оставил относительный порядок Хатлонская > Согдийская > РРП > ГБАО ===
yield_mult_fixed = {
    "Хатлонская": 1.20,
    "Согдийская": 1.05,
    "РРП":        0.75,   # ↓ было 0.95
    "ГБАО":       0.20    # ↓ было 0.50
}

# === Торговля (импорт/экспорт bias) ===
trade_bias = {
    "Хатлонская": {"imp": 0.9, "exp": 1.1},
    "Согдийская": {"imp": 1.0, "exp": 1.0},
    "РРП":        {"imp": 1.5, "exp": 0.5},
    "ГБАО":       {"imp": 1.3, "exp": 0.1}
}

# === Цены: региональный коэффициент ===
# Изменил порядок и коэффициенты так, чтобы в результирующей цене
# получилось: Хатлонская > Согдийская > РРП > ГБАО.
price_region_bias = {
    "Хатлонская": 1.25,
    "Согдийская": 1.10,
    "РРП":       0.49,
    "ГБАО":      0.21
}

# === Базовый спрос (т/мес) и сезонный коэффициент ===
base_demand = {
    "Пшеница": 30000,
    "Картофель":20000,
    "Овощи":    25000,
    "Мясо":     15000,
    "Молоко":   40000
}


season_factor = {
    "Январь": 0.9,
    "Февраль": 0.9,
    "Март": 1.0,
    "Апрель": 1.1,
    "Май": 1.2,
    "Июнь": 1.3,
    "Июль": 1.3,
    "Август": 1.2,
    "Сентябрь": 1.1,
    "Октябрь": 1.0,
    "Ноябрь": 0.95,
    "Декабрь": 0.9
}

# === Региональные верхние границы реалистичной самодостаточности (по замечанию) ===
# Автор хотел: Хатлон ≈ до 60%, Согдийская ≈ до 57%, РРП ≈ до 25%, ГБАО ≈ до 5%.
region_selfsuff_cap = {
    "Хатлонская": 60.0,
    "Согдийская": 57.0,
    "РРП": 25.0,
    "ГБАО": 5.0
}

rows = []

for year in years:
    wheat_area_country   = np.random.uniform(*country_targets["Пшеница"]["area"])
    wheat_yield_country  = np.random.uniform(*country_targets["Пшеница"]["yield"])
    veg_area_country     = np.random.uniform(*country_targets["Овощи"]["area"])
    veg_yield_country    = np.random.uniform(*country_targets["Овощи"]["yield"])
    pot_area_country     = np.random.uniform(*country_targets["Картофель"]["area"])
    pot_yield_country    = np.random.uniform(*country_targets["Картофель"]["yield"])
    meat_country         = np.random.uniform(*country_targets["Мясо"]["volume"])
    milk_country         = np.random.uniform(*country_targets["Молоко"]["volume"])

    def with_noise(x): return x * np.random.uniform(0.95, 1.05)

    area_by_region = {
        "Пшеница":   {r: with_noise(wheat_area_country   * shares["Пшеница"][r])   for r in regions},
        "Овощи":     {r: with_noise(veg_area_country     * shares["Овощи"][r])     for r in regions},
        "Картофель": {r: with_noise(pot_area_country     * shares["Картофель"][r]) for r in regions},
    }

    vol_by_region = {
        "Мясо":   {r: with_noise(meat_country   * shares["Мясо"][r])   for r in regions},
        "Молоко": {r: with_noise(milk_country   * shares["Молоко"][r]) for r in regions},
    }

    for region, rinfo in regions.items():
        # === Растениеводство ===
        for product in ["Пшеница","Картофель","Овощи"]:
            annual_area = area_by_region[product][region]
            months_h = harvest_months[product]
            n_harvest = max(1, len(months_h))
            for month in months:
                is_harvest = month in months_h
                area = (annual_area / n_harvest) if is_harvest else 0.0
                base_yield_reg = {
                    "Пшеница": wheat_yield_country,
                    "Картофель": pot_yield_country,
                    "Овощи": veg_yield_country
                }[product] * yield_mult_fixed[region]

                yield_month = base_yield_reg if is_harvest else 0.0
                production = area * yield_month

                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * np.random.uniform(0.9, 1.1)

                # === Баланс и торговля (обновлённая логика) ===
                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.6, 0.95) * trade_bias[region]["imp"]
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.3, 0.9) * trade_bias[region]["exp"]

                # === Самодостаточность ===
                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                # Применяем региональную реалистичную верхнюю границу
                self_suff = min(region_selfsuff_cap[region], self_suff)

                base_p = np.random.uniform(*base_prices[product])
                if production <= 0.0:
                    price = 0.0
                else:
                    if is_harvest:
                        d_s_ratio = consumption / (production + imp + eps)
                        price = (base_p * (1 + price_alpha[product] * (d_s_ratio - 1))
                                 + np.random.normal(0, 0.25)) * price_region_bias[region]
                    else:
                        price = (base_p + np.random.normal(0, 0.25)) * price_region_bias[region]

                price = max(0.0, price)

                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.2)
                rain = month_rain_shift[month] + np.random.normal(0, 8)

                rows.append([
                    year, month, region, product,
                    round(area,1), round(yield_month,2), round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

        # === Животноводство ===
        for product in ["Мясо","Молоко"]:
            annual_prod = vol_by_region[product][region]
            for month in months:
                production = annual_prod / 12.0
                consumption = base_demand[product] * shares[product][region] \
                              * season_factor[month] * np.random.uniform(0.9, 1.1)

                # === Баланс и торговля (обновлённая логика) ===
                net_need = consumption - production
                if net_need > 0:
                    imp = net_need * np.random.uniform(0.6, 0.95) * trade_bias[region]["imp"]
                    exp = 0.0
                else:
                    imp = 0.0
                    exp = (-net_need) * np.random.uniform(0.3, 0.9) * trade_bias[region]["exp"]

                eps = 1e-6
                self_suff = ((production + imp) / (consumption + eps)) * 100
                self_suff = min(region_selfsuff_cap[region], self_suff)

                base_p = np.random.uniform(*base_prices[product])
                d_s_ratio = consumption / (production + imp + eps)
                price = (base_p * (1 + price_alpha[product] * (d_s_ratio - 1))
                         + np.random.normal(0, 0.35)) * price_region_bias[region]
                price = max(0.0, price)

                temp = np.random.uniform(*rinfo["temp"]) + month_temp_shift[month] + np.random.normal(0, 1.2)
                rain = month_rain_shift[month] + np.random.normal(0, 8)

                rows.append([
                    year, month, region, product,
                    0.0, 0.0, round(production,1),
                    round(price,2), round(imp,1), round(exp,1),
                    round(consumption,1), round(self_suff,1),
                    round(temp,1), round(rain,1), rinfo["zone"]
                ])

# === Формируем итоговый DataFrame ===
df = pd.DataFrame(rows, columns=[
    "Год","Месяц","Регион","Продукт",
    "Площадь (га)","Урожайность (т/га)","Производство (т)",
    "Цена (сом/т)","Импорт (т)","Экспорт (т)",
    "Потребление (т)","Самодостаточность (%)",
    "Температура (°C)","Осадки (мм)","Зона"
])

# === Производные бинарные признаки ===
q75 = df["Урожайность (т/га)"].quantile(0.75)
df["Высокая урожайность"] = np.where(
    (df["Урожайность (т/га)"] >= q75) & (df["Температура (°C)"] >= df["Температура (°C)"].median()),
    1, 0
)

if __name__ == '__main__':
    display(df.head(10))

,Год,Месяц,Регион,Продукт,Площадь (га),Урожайность (т/га),Производство (т),Цена (сом/т),Импорт (т),Экспорт (т),Потребление (т),Самодостаточность (%),Температура (°C),Осадки (мм),Зона,Высокая урожайность
0,2023,Январь,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,6100.7,0.0,10999.6,55.5,9.0,27.7,Равнинная,0
1,2023,Февраль,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,7599.5,0.0,11197.9,60.0,15.9,25.2,Равнинная,0
2,2023,Март,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,8127.8,0.0,11548.1,60.0,16.7,61.4,Равнинная,0
3,2023,Апрель,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,7289.2,0.0,12113.6,60.0,21.5,74.2,Равнинная,1
4,2023,Май,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,10011.9,0.0,15346.8,60.0,31.8,66.9,Равнинная,1
5,2023,Июнь,Хатлонская,Пшеница,47471.5,4.16,197415.1,5542.07,0.0,179726.0,14272.6,60.0,38.4,33.3,Равнинная,1
6,2023,Июль,Хатлонская,Пшеница,47471.5,4.16,197415.1,5138.82,0.0,133953.6,16732.9,60.0,31.3,20.4,Равнинная,1
7,2023,Август,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,11157.8,0.0,15061.3,60.0,35.2,19.8,Равнинная,1
8,2023,Сентябрь,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,9955.3,0.0,13888.5,60.0,27.8,46.6,Равнинная,1
9,2023,Октябрь,Хатлонская,Пшеница,0.0,0.00,0.0,0.00,6234.1,0.0,10861.0,57.4,26.5,34.1,Равнинная,1


In [2]:
df['Высокая урожайность'].value_counts()

Высокая урожайность
0    240
1    240
Name: count, dtype: int64

In [3]:
df

,Год,Месяц,Регион,Продукт,Площадь (га),Урожайность (т/га),Производство (т),Цена (сом/т),Импорт (т),Экспорт (т),Потребление (т),Самодостаточность (%),Температура (°C),Осадки (мм),Зона,Высокая урожайность
0,2023,Январь,Хатлонская,Пшеница,0.0,0.0,0.0,0.00,6100.7,0.0,10999.6,55.5,9.0,27.7,Равнинная,0
1,2023,Февраль,Хатлонская,Пшеница,0.0,0.0,0.0,0.00,7599.5,0.0,11197.9,60.0,15.9,25.2,Равнинная,0
2,2023,Март,Хатлонская,Пшеница,0.0,0.0,0.0,0.00,8127.8,0.0,11548.1,60.0,16.7,61.4,Равнинная,0
3,2023,Апрель,Хатлонская,Пшеница,0.0,0.0,0.0,0.00,7289.2,0.0,12113.6,60.0,21.5,74.2,Равнинная,1
4,2023,Май,Хатлонская,Пшеница,0.0,0.0,0.0,0.00,10011.9,0.0,15346.8,60.0,31.8,66.9,Равнинная,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2024,Август,РРП,Молоко,0.0,0.0,19520.9,4093.39,0.0,2808.2,10485.0,25.0,32.3,15.8,Разнообразная,1
476,2024,Сентябрь,РРП,Молоко,0.0,0.0,19520.9,4107.67,0.0,4738.7,8407.5,25.0,25.9,39.4,Разнообразная,1
477,2024,Октябрь,РРП,Молоко,0.0,0.0,19520.9,4335.39,0.0,4046.3,7335.3,25.0,25.5,55.0,Разнообразная,1
478,2024,Ноябрь,РРП,Молоко,0.0,0.0,19520.9,3753.27,0.0,4727.6,8151.8,25.0,21.2,47.0,Разнообразная,1


In [4]:
# === Сохранение ===
df.to_csv("tajikistan_agriculture_2023_2024.csv", index=False, encoding="utf-8-sig")

print("✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024.csv'")

✅ Синтетический датасет успешно создан и сохранён как 'tajikistan_agriculture_2023_2024.csv'


In [5]:
print(df.head(7))

    Год    Месяц      Регион  Продукт  Площадь (га)  Урожайность (т/га)  \
0  2023   Январь  Хатлонская  Пшеница           0.0                0.00   
1  2023  Февраль  Хатлонская  Пшеница           0.0                0.00   
2  2023     Март  Хатлонская  Пшеница           0.0                0.00   
3  2023   Апрель  Хатлонская  Пшеница           0.0                0.00   
4  2023      Май  Хатлонская  Пшеница           0.0                0.00   
5  2023     Июнь  Хатлонская  Пшеница       47471.5                4.16   
6  2023     Июль  Хатлонская  Пшеница       47471.5                4.16   

   Производство (т)  Цена (сом/т)  Импорт (т)  Экспорт (т)  Потребление (т)  \
0               0.0          0.00      6100.7          0.0          10999.6   
1               0.0          0.00      7599.5          0.0          11197.9   
2               0.0          0.00      8127.8          0.0          11548.1   
3               0.0          0.00      7289.2          0.0          12113.6   
4   

In [6]:
import plotly.graph_objects as go

# Данные
products = ['Картофель', 'Молоко', 'Мясо', 'Овощи', 'Пшеница']
values_2023 = [1.377, 1.057, 0.204, 4.117, 1.619]
values_2024 = [1.656, 0.964, 0.212, 3.734, 1.393]

# График
fig = go.Figure()

fig.add_trace(go.Bar(
    x=products,
    y=values_2023,
    name='2023',
    marker_color='rgb(158,202,225)'
))

fig.add_trace(go.Bar(
    x=products,
    y=values_2024,
    name='2024',
    marker_color='rgb(255,153,102)'
))

# Настройки
fig.update_layout(
    title='📊 Производство сельхозпродукции в Таджикистане (млн тонн)',
    xaxis_title='Продукт',
    yaxis_title='Объём производства (млн тонн)',
    barmode='group',
    template='plotly_white',
    legend=dict(x=0.85, y=1.15),
    height=500
)

fig.show()
